In [194]:
import numpy as np 
import math

In [195]:
class Activition_functions():

    @staticmethod
    def Tanh(z):
        return (math.exp(z) - math.exp(-z)) / (math.exp(z) + math.exp(-z))
     
    def softmax(vector):
        e = np.exp(vector)
        return e / e.sum()

In [196]:
W_H = np.array([[0.1 ,0.5 , 0.1],[0.5,0.9,0.2],[0.1,0.3,0.1]])

W_X = np.array([[0.6,0.8,0.4,0.8],[0.2,0.2,0.8,0.7],[0.9,0.8,0.1,0.2]])

W_Y = np.array([[0.9,0.8,0.3],[0.2,0.3,0.4],[0.6,0.9,0.1],[0.5,0.0,0.3]])

v = np.dot(W_H,np.zeros((3,1))) + np.dot(W_X,np.array([1,0,0,0]).reshape((4,1)))

c = np.array([Activition_functions.Tanh(float(x))  for x in list(v)]).reshape((3,1))


Activition_functions.softmax(np.dot(W_Y,c))

C:\Users\Computec\AppData\Local\Temp\ipykernel_20288\3150391295.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  c = np.array([Activition_functions.Tanh(float(x))  for x in list(v)]).reshape((3,1))


array([[0.321599  ],
       [0.2149301 ],
       [0.24193486],
       [0.22153605]])

In [197]:
class RNN():
    def __init__(self,word:str,k:int,d:int,W_H: np.array = None, W_X: np.array = None, W_Y: np.array = None):
        self.k =k
        self.d =d
        self.word = word
        self.W_H = np.random.uniform(-1, 1, size=(self.d, self.d)) if W_H is None else W_H
        self.W_X = np.random.uniform(-1, 1, size=(self.d, self.k)) if W_X is None else W_X
        self.W_Y = np.random.uniform(-1, 1, size=(self.k, self.d)) if W_Y is None else W_Y
        self.h = {"t1":np.zeros((self.d,1))}
        self.letter = self.__one_hot_list()
        

    def __one_hot_list(self,):
        unique_chars = self.word
        char_to_index = {char: idx for idx, char in enumerate(unique_chars)}
        one_hot_vectors = []
        for char in self.word:
            vector = [0] * len(unique_chars)  
            vector[char_to_index[char]] = 1 
            one_hot_vectors.append(vector)    
        return one_hot_vectors
    
    
    def Rnn_forward(self):
        for x in range(len(self.letter)):
            first_step = np.dot(self.W_H,self.h[f"t{x+1}"]) + np.dot(self.W_X,np.array(self.letter[x]).reshape((4,1)))
            second_step = np.array([Activition_functions.Tanh(float(x))  for x in list(first_step)]).reshape((3,1))
            self.h[f"t{x+1+1}"] = second_step
        return Activition_functions.softmax(np.dot(self.W_Y,second_step))

In [198]:
rnn = RNN("dogs",k=4,d=3,W_H=W_H,W_X=W_X,W_Y=W_Y)
rnn.Rnn_forward()

C:\Users\Computec\AppData\Local\Temp\ipykernel_20288\1360227668.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  second_step = np.array([Activition_functions.Tanh(float(x))  for x in list(first_step)]).reshape((3,1))


array([[0.41076952],
       [0.14419494],
       [0.31164274],
       [0.1333928 ]])